In [10]:
import cv2
import numpy as np

contador = 300
cap = cv2.VideoCapture(0)
while cap.isOpened():
    contador = contador - 1
    if contador < 7:
        contador =7
    contador_final =  round(400 / contador)

    ret, frame = cap.read()
    if not ret:
        print("Error al capturar el video")
        break

    escala = 0.5  # Ajusta esta escala si la imagen es muy grande
    imagen_reducida = frame#cv2.resize(frame, (int(frame.shape[1] * escala), int(frame.shape[0] * escala)))

    # Convertir la imagen a escala de grises
    gris = cv2.cvtColor(imagen_reducida, cv2.COLOR_BGR2GRAY)

    # Aplicar el detector de bordes Canny para detectar contornos
    bordes = cv2.Canny(gris, 100, 200)

    # Encontrar todos los contornos
    contornos, _ = cv2.findContours(bordes, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una máscara vacía del tamaño de la imagen para el aura
    mascara_aura = np.zeros_like(imagen_reducida)

    # Definir un color para el aura
    color_aura = (0, 255, 255)  # Amarillo brillante para el aura

    # Dibujar todos los contornos en la máscara con mayor grosor
    grosor_aura = contador_final  # Aumenta este valor para un aura más pronunciada
    for contorno in contornos:
        cv2.drawContours(mascara_aura, [contorno], -1, color_aura, grosor_aura)

    # Aplicar un desenfoque Gaussiano a la máscara para suavizar el aura
    mascara_aura = cv2.GaussianBlur(mascara_aura, (51, 51), 0)

    # Combinar la imagen original con la máscara del aura
    resultado = cv2.addWeighted(imagen_reducida, 1, mascara_aura, 0.5, 0)
    # Mostrar la imagen original y el resultado
    cv2.imshow('Imagen Original', frame)
    cv2.imshow('Colores Destacados (Aura)', resultado)

    # Romper el bucle si se presiona la tecla 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()


In [11]:
import cv2
import numpy as np
import mediapipe as mp




cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error al capturar el video")
        break

    # Inicializar MediaPipe Selfie Segmentation
    mp_selfie_segmentation = mp.solutions.selfie_segmentation
    segmentador = mp_selfie_segmentation.SelfieSegmentation(model_selection=1)

    # Cargar la imagen
    #frame = cv2.imread('persona.jpg')
    escala = 1  # Escalar la imagen si es muy grande
    imagen_reducida = cv2.resize(frame, (int(frame.shape[1] * escala), int(frame.shape[0] * escala)))

    # Convertir la imagen a RGB porque MediaPipe usa imágenes en formato RGB
    imagen_rgb = cv2.cvtColor(imagen_reducida, cv2.COLOR_BGR2RGB)

    # Realizar la segmentación
    resultado = segmentador.process(imagen_rgb)

    # Crear una máscara binaria a partir de la segmentación (1 es persona, 0 es fondo)
    mascara_segmentacion = resultado.segmentation_mask > 0.5  # Umbral de segmentación

    # Convertir la máscara binaria a una imagen de 8 bits para poder usarla con OpenCV
    mascara_segmentacion = np.uint8(mascara_segmentacion * 255)

    # Crear una máscara vacía del tamaño de la imagen para el aura
    mascara_aura = np.zeros_like(imagen_reducida)

    # Definir un color para el aura
    color_aura = (0, 255, 255)  # Amarillo brillante para el aura

    # Encontrar los contornos de la máscara de segmentación
    contornos, _ = cv2.findContours(mascara_segmentacion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Dibujar el contorno de la persona con mayor grosor para crear el aura
    grosor_aura = 30  # Ajustar el grosor para un aura más pronunciada
    for contorno in contornos:
        cv2.drawContours(mascara_aura, [contorno], -1, color_aura, grosor_aura)

    # Aplicar un desenfoque Gaussiano a la máscara para suavizar el aura
    desenfoque_aura = (101, 101)  # Aumenta el desenfoque para suavizar el aura
    mascara_aura = cv2.GaussianBlur(mascara_aura, desenfoque_aura, 0)

    # Combinar la imagen original con la máscara del aura
    resultado_final = cv2.addWeighted(imagen_reducida, 1, mascara_aura, 0.6, 0)

    # Mostrar el resultado
    cv2.imshow('Imagen Original', imagen_reducida)
    cv2.imshow('Aura con Selfie Segmentation', resultado_final)

    # Romper el bucle si se presiona la tecla 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Esperar y cerrar las ventanas
#cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()


In [8]:
cap.release()
cv2.destroyAllWindows()
